# 1 - Introduction et problématique

petite intro des familles toi meme tu sais

# 2 - Récupération et traitement des données

Cette section a pour objectif de constituer la base de donnée qui sera assujettie à l'ensemble de nos analyses.
Pour commencer, nous pouvons nous rendre sur le site officiel de la IMDB : https://www.imdb.com/
Sur le haut de la page, il est possible d'effectuer une recherche personnalisée afin d'obtenir une liste très étendue de films :
<img src="image1.png">

Une fois la recherche lancée, le site nous redirige vers le début de la liste souhaitée :
<img src="image2.png">

On peut observer plusieures informations : 447 167 films sont à notre disposition, avec 250 films par page. De plus pour chaque film, on a accès à plusieurs informations : son titre, année de sortie, durée, genres principaux, note du public, nombre de votes, metascore, directeurs, acteurs. 
L'objectif est de constituer notre base de données en regroupant dans un dataframe toutes ces informations pour chaque film. 

Et rien de plus essentiel pour effectuer cette tâche que de mettre en place une belle opération de web scraping ! Moteur, et ... Action !

## 2.1 - Création d'une base IMDB web scrapée

Avant toute chose, il est important d'importer les modules qui nous seront indispensables :

In [5]:
# Module pandas pour manipuler le dataframe ensuite constitué :
import pandas as pd

# Module numpy pour manipuler essentiellement des arrays :
import numpy as np

# Module re (regex ou regular expressions) pour le nettoyage de la base :
import re

# Modules pour le web scraping :
import urllib
import bs4
# pour la récupération du code source :
from urllib import request

Ensuite on crée les tableaux vides pour les remplir des informations par la suite :

In [2]:
tconst_list = []
title_list =[]
year_list = []
runtime_list = []
genres_list = []
metascore_list = []
rate_list = []
votes_list = []
certificate_list = []
director_list = []
casting_list = []

Ensuite, on prend l'url de la toute première page (début de la liste des films à scrapper), puis on l'insère dans une liste. Ceci nous permettra de lancer l'instruction de la première page à scrapper au tout début de la boucle. Puis dans la boule on récoltera les liens des pages suivantes qu'on stockera également dans cette liste :

In [3]:
#URL à mettre à jour à chaque session de scraping
url = "https://www.imdb.com/search/title/?title_type=feature&adult=include&count=250&after=Wzc0MjQxMywidHQyMjk0NjY1IiwyNzUwMDFd&ref_=adv_nxt"
list_url = [url]

Comme il y a près de 450000 films à web scrapper et qu'il y a 250 films par pages, nous avons environ N = 450000/250 = 1800 pages à web scrapper.

Comme le nombre de films à scrapper est conséquent, la boucle de web scraping est également très longue à s'executer. Par motif de précaution, nous avons décidé de fragmenter la boucle : plutôt que de lancer une seule boucle de N = 1800 pages à scrapper, nous lancerons 6 boucles de 300 pages à web scrap. A la fin de chaque boucle, nous prendrons le dernier url scrappé, qui sera l'url de départ de la boucle qui suivra.
A la fin de chaque boucle, nous enregistrerons chaque lot de données dans une base pickle et une autre base au format csv. A la fin, nous n'aurons plus qu'à importer nos 6 bases csv, puis effectuer une jointure, et nous obtiendrons notre dataframe d'environ 450 000 observations

A titre de démonstration, nous nous contenterons de lancer ici uniquement une boucle pour web scraper 1 page, pour montrer que la boucle marche bien sans avoir à attendre une éternité.

In [4]:
n = 3    # Nombre de pages à scraper pour une étape

for i in range(n): # Lancement de la grande boucle de web scraping
    
    url = list_url[-1]     # Si premier scraping, on prend l'url définie plus haut,  
                            # sinon on prend l'url récupérée sur la dernière page scrapée

    # Etape de récupération du code source :
    req = urllib.request.Request(url, headers = {'User-Agent' : 'Mozilla/5.0'})
    request_text = request.urlopen(req).read()

    # Utilisation du package BeautifulSoup 
    # pour interpréter les balises contenues dans la chaine de caractères du code source
    page = bs4.BeautifulSoup(request_text, "html.parser")

    # Dans l'architecture du code source, toutes les informations qu'on veut receuillir 
    # pour un seul film sont contenues dans une division ayant la classe "lister-item mode-advanced"
    # Donc on récolte toutes ces divisions pour pouvoir ensuite aller récolter pour tous les films de 
    # la page concernée
    divisions = page.findAll('div', {'class' : 'lister-item mode-advanced'})

    # On boucle pour tous les films de la page :
    for division in divisions :
        
        # Récolte du titre du film et stockage dans le tableau title_list :
        title = division.h3.a.text
        title_list.append(title)

        # Récolte de l'année de sortie du film et stockage dans le tableau year_list :
        year = division.h3.find('span',{'class': "lister-item-year text-muted unbold"}).text
        year_list.append(year)

        # Récolte de la durée du film et stockage dans le tableau runtime_list :
        runtime = division.find('span',{'class':'runtime'}).text if division.p.find('span',{'class':'runtime'}) else 'NA'
        runtime_list.append(runtime)

        # Récolte des genres principaux du film et stockage dans le tableau genres_list :
        genres = division.find('span',{'class':'genre'}).text if division.p.find('span',{'class':'genre'}) else 'NA'
        genres_list.append(genres)

        # Récolte du métascore du film et stockage dans le tableau metascore_list :
        metascore = division.find('span', {"class" : 'metascore'}).text if division.find('span', {'class':'metascore'}) else 'NA'
        metascore_list.append(metascore)

        # Récolte de la note du film et stockage dans le tableau rate_list :
        rate = division.find('div',{'class':'inline-block ratings-imdb-rating'}).find('strong').text if division.find('div',{'class':'inline-block ratings-imdb-rating'}) else 'NA'
        rate_list.append(rate)

        # Récolte du nombre de votes pour le film et stockage dans le tableau votes_list :
        votes = division.find('span',{'name':'nv'}).text if division.find('span',{'name':'nv'}) else 'NA'
        votes_list.append(votes)

        # Récolte de l'information s'il s'agit d'un film pour adultes et stockage dans le tableau certificate_list : 
        certificate = division.find('span',{'class':'certificate'}).text if division.find_all('span',{'class':'certificate'}) else 'NA'
        certificate_list.append(certificate)

        # Récolte du réalisateur du film et stockage dans le tableau director_list :
        director = division.find_all('p',class_ ='')[0].a.text if division.find_all('p',class_ ='')[0].a else 'NA'
        director_list.append(director)

        # Récolte du casting (acteurs) du film et stockage dans le tableau casting_list :
        casting = [acteur.text for acteur in division.find_all('p',class_ ='')[0].find_all('a')[1:]]
        casting_list.append(casting)

        # Récolte du tconst du film (identifiant du film) et stockage dans le tableau tconst_list :
        txt = str(division.h3.a)
        expression = re.compile("tt\d+")        
        tconst = expression.findall(txt)
        tconst_list.append(tconst)
        
    # Crawling : récupération de l'url de la page suivante
    next_url = "https://www.imdb.com"+ page.find('div',{'class' : 'desc'}).find('a',{'class':'lister-page-next next-page'})['href']
    list_url.append(next_url)
    print("nombre de pages scrappées :" + str(i+1), end = "\r")

Une fois la boucle terminée, on peut simplement s'assurer que tout s'est bien déroulé en vérifiant que chaque liste a la même taille :

In [5]:
print(len(tconst_list), len(title_list),len(year_list),
    len(runtime_list),len(genres_list),len(metascore_list), len(rate_list),
    len(votes_list), len(certificate_list), len(director_list),len(casting_list))

250 250 250 250 250 250 250 250 250 250 250


Ensuite, on peut former un pandas DataFrame de la liste de films qui vient d'être web scrapée :

In [6]:
df = pd.DataFrame({'tconst' :tconst_list,
                  'title': title_list,
                  'year' : year_list,
                  'runtime': runtime_list,
                  'genres': genres_list,
                  'metascore' : metascore_list,
                  'rate' : rate_list,
                  'votes' : votes_list,
                  'certificate' : certificate_list,
                  'director' : director_list,
                  'casting' : casting_list
                  })

Puis on affiche la base brute ainsi formée :

In [7]:
df

,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
0,[tt2829268],E Som Somwang: Cha Cha Cha,(2009),NA,\nComedy,NA,3.8,6,NA,Vorawit Phonginsee,"[Annie Brooks, Kom Chauncheun, Note Chern-Yim,..."
1,[tt11201904],Dynafit 19 SS,(2019),NA,\nSport,NA,NA,NA,NA,Jung-kyu Kim,"[Britni Camacho, Jeris Dupree, Ryan Klarenbach..."
2,[tt1342377],Fa da zhi ren,(1979),NA,\nDrama,NA,NA,NA,NA,Ning Tu,"[Hao Chen, Wen Hsia, Pao-Shu Kao, Chun-Hsiung Ko]"
3,[tt4082144],The Ashes of CHIKARA,(2014),77 min,"\nAction, Drama",NA,NA,NA,NA,Ian Vaflor,[]
4,[tt0231305],Bus Conductor,(1959),NA,\nDrama,NA,NA,NA,NA,Dwarka Khosla,"[Shyama, Prem Nath, Amarnath, Maruti]"
...,...,...,...,...,...,...,...,...,...,...,...
245,[tt0306683],Cousteau: Alaska: Outrage at Valdez,(1989),48 min,NA,NA,8.8,19,NA,Jean-Michel Cousteau,[]
246,[tt1538384],Sitamgar,(1967),NA,NA,NA,NA,NA,NA,Aziz Merathi,"[Darpan, Yusuf Khan, Sabiha Khanum, Santosh Ku..."
247,[tt1908463],Bengaloored,(2010),NA,"\nDrama, Romance",NA,8.0,29,NA,Swaroop Kanchi,"[Harish Raj, Meghana Mudiyam, Srinivasa Prabhu..."
248,[tt2996906],Kimmy Dora: Ang kiyemeng prequel,(2013),97 min,\nComedy,NA,6.3,26,12,Chris Martinez,"[Eugene Domingo, Sam Milby, Joel Torre, Angel ..."


Il ne nous reste plus qu'à enregistrer notre dataframe en pickle et en csv :

In [ ]:
df.to_pickle("./data/df_250000_275000.pkl")

In [ ]:
df.to_csv("df_250000_275000", sep='\t', index=False)

Et on récupère l'avant dernier lien de la liste qui servira d'url de départ de la prochaine boucle (on prend bien soin de partir de l'avant dernier pour qu'entre deux bases consécutives on ait un film en commun pour faire les jointures par la suite.

In [ ]:
list_url[-2]

On procède aux jointures:

In [88]:
liste_subbase = []
for i in range(1,21):
    df_temp = pd.read_csv('subdatas/subbase' +str(i), sep = '\t', header = 0, low_memory = False)
    liste_subbase.append(df_temp)

len(liste_subbase)

20

In [93]:
df = pd.concat([subbase for subbase in liste_subbase])
len(df)
# C'est bien concatenate
df
df.reset_index()

,index,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
0,0,['tt13143964'],Borat: Subsequent Moviefilm,(2020),95 min,\nComedy,68.0,7.0,"50,191",18,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,1,['tt1070874'],Les Sept de Chicago,(2020),129 min,"\nDrama, History, Thriller",76.0,7.9,"37,295",16,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,2,['tt2235695'],Rebecca,(2020),121 min,"\nDrama, Mystery, Romance",46.0,6.0,"11,829",13,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."
3,3,['tt10682266'],Hubie Halloween,(2020),102 min,"\nComedy, Fantasy, Mystery",53.0,5.2,"27,526",7,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',..."
4,4,['tt2222042'],Love and Monsters,(2020),109 min,"\nAction, Adventure, Comedy",59.0,7.1,"10,780",PG-13,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
456092,6092,['tt13366978'],Maria,(X),NaN,\nComedy,NaN,NaN,NaN,NaN,Alec Pronovost,"['Florence Longpré', 'Mariana Mazza', 'Alice P..."
456093,6093,['tt13367186'],Kill Joy,NaN,NaN,"\nAction, Crime, Drama",NaN,NaN,NaN,NaN,NaN,[]
456094,6094,['tt13367214'],Covid Plus,NaN,NaN,\nDrama,NaN,NaN,NaN,NaN,NaN,[]
456095,6095,['tt13367330'],Muralla china,(2020),NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [119]:
# save de la base finale en pickle :
df.to_pickle("/Users/UPTONHENRI/Desktop/data/data.pkl")

In [120]:
# save de la base finale en csv :
df.to_csv("/Users/UPTONHENRI/Desktop/data/data.csv", sep='\t', index=False)

# Cleaning du DataFrame global

In [97]:
pd.options.mode.chained_assignment = None 
df["year"] = df["year"].astype(str)
df["runtime"] = df["runtime"].astype(str)
df["genres"] = df["genres"].astype(str)

In [98]:
df["tconst"] = df["tconst"].apply(lambda x: x[0])
df["year"] = df["year"].apply(lambda x :x.replace('(','').replace(')',''))
df["runtime"] = df["runtime"].apply(lambda x :x.replace(' min',''))
df["genres"] = df["genres"].apply(lambda x :x.replace("\n",""))

In [14]:
df = pd.read_csv("./data/data.csv", sep="\t")

In [15]:
df.loc[df['title'] == "Forrest Gump"]

,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
181,tt0109830,Forrest Gump,1994,142,"Drama, Romance",82.0,8.8,"1,771,389",Tous publics,Robert Zemeckis,"['Tom Hanks', 'Robin Wright', 'Gary Sinise', '..."


In [16]:
df

,tconst,title,year,runtime,genres,metascore,rate,votes,certificate,director,casting
0,tt13143964,Borat: Subsequent Moviefilm,2020,95,Comedy,68.0,7.0,"50,191",18,Jason Woliner,"['Sacha Baron Cohen', 'Maria Bakalova', 'Tom H..."
1,tt1070874,Les Sept de Chicago,2020,129,"Drama, History, Thriller",76.0,7.9,"37,295",16,Aaron Sorkin,"['Eddie Redmayne', 'Alex Sharp', 'Sacha Baron ..."
2,tt2235695,Rebecca,2020,121,"Drama, Mystery, Romance",46.0,6.0,"11,829",13,Ben Wheatley,"['Lily James', 'Armie Hammer', 'Kristin Scott ..."
3,tt10682266,Hubie Halloween,2020,102,"Comedy, Fantasy, Mystery",53.0,5.2,"27,526",7,Steven Brill,"['Adam Sandler', 'Kevin James', 'Julie Bowen',..."
4,tt2222042,Love and Monsters,2020,109,"Action, Adventure, Comedy",59.0,7.1,"10,780",PG-13,Michael Matthews,"[""Dylan O'Brien"", 'Jessica Henwick', 'Michael ..."
...,...,...,...,...,...,...,...,...,...,...,...
456092,['tt13366978'],Maria,X,NaN,Comedy,NaN,NaN,NaN,NaN,Alec Pronovost,"['Florence Longpré', 'Mariana Mazza', 'Alice P..."
456093,['tt13367186'],Kill Joy,NaN,NaN,"Action, Crime, Drama",NaN,NaN,NaN,NaN,NaN,[]
456094,['tt13367214'],Covid Plus,NaN,NaN,Drama,NaN,NaN,NaN,NaN,NaN,[]
456095,['tt13367330'],Muralla china,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]


In [28]:
pd.options.mode.chained_assignment = None
df["tconst"][13764:456096] = df["tconst"][13764:456096].replace("'",'').replace('[','').replace(']','')

In [37]:
df["tconst"][13764]

'tt4144190'

In [11]:
df["tconst"] = df["tconst"].apply(lambda x: x[0])

In [34]:
df["tconst"][456096]

"['tt13367334']"

In [40]:
type(df['tconst'][100000])

str